# Building Fast Queries on a CSV

We will imagine that we own an online laptop store and want to build a way to answer a few different business questions about our inventory.

In general, the task is to create a class to search for specific information in the selected file. Next, measure its performance and then refine it so that it works faster.

At first we need to create a `class which saves a csv file in appropriate form` for further analysis.  
Separating the header and rows and putting them into **self.variables**.

In [3]:
import csv

class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as open_f:
            file = list(csv.reader(open_f))
        self.header = file[0]
        self.rows = file[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
            
laptops = Inventory('laptops.csv')
print(f'Header:\n{laptops.header}\nLength of file:\n{len(laptops.rows)}\nFirst 5 rows:')
for row in laptops.rows[:5]:
    print(row)

Header:
['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
Length of file:
1303
First 5 rows:
['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', 1339]
['7287764', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', 898]
['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
['9722156', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', 2537]
['8550527', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel C

# Getting laptop info from ID

Next we adding a **get_laptop_from_id** method.  
A method returns the whole row with laptop information by passing it an ID string. The laptop info stored in **laptops.rows**.

In [4]:
class Inventory():
    
    def __init__(self, csv_filename): # 'csv_filename.csv'
        with open(csv_filename) as open_f:
            file = list(csv.reader(open_f))
        self.header = file[0]
        self.rows = file[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
        
    def get_laptop_from_id(self, laptop_id): # 'laptop_id'
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
            
laptops = Inventory('laptops.csv')
print(laptops.get_laptop_from_id('3362737'))
print(laptops.get_laptop_from_id('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


# Improving ID lookups

The `previous decision` has `time complexity O(R)`, where R - is the number of rows in a file (our file is list of lists).

`Here` is implementation of `O(1) time complexity` - by creating a dictionary for faster lookup instead of storing our laptops data set in a list (`self.id_to_row, dict vs self.rows, list`).

In [5]:
class Inventory():
    
    def __init__(self, csv_filename): # 'csv_filename.csv'
        with open(csv_filename) as open_f:
            file = list(csv.reader(open_f))
        self.header = file[0]
        self.rows = file[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
        
    def get_laptop_from_id(self, laptop_id): # 'laptop_id'
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id): # 'laptop_id'
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
            
laptops = Inventory('laptops.csv')
print(laptops.get_laptop_from_id_fast('3362737'))
print(laptops.get_laptop_from_id_fast('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


# Comparing the Performance

Let's compare the `execution speed` of our lookup methods - **get_laptop_from_id** and **get_laptop_from_id_fast**.  
First step is to `generate the ids object with 10.000 queries` with string ID (in ID range) using random module.  
Second step is to `measure the execution time` by using time module.

In [6]:
import time, random

ids = [str(random.randint(1000000, 9999999)) for _ in range(10000)]

total_time_no_dict = 0
for each_id in ids:
    start = time.time()
    laptops.get_laptop_from_id(each_id)
    end = time.time()
    total_time_no_dict += end - start
    
total_time_dict = 0
for each_id in ids:
    start = time.time()
    laptops.get_laptop_from_id_fast(each_id)
    end = time.time()
    total_time_dict += end - start
    
print(f'Lookup time without dictionary: {round(total_time_no_dict, 5)}\nLookup time with dictionary: {round(total_time_dict, 5)}')

Lookup time without dictionary: 1.34343
Lookup time with dictionary: 0.00538


The `method with generating a dictionary` first is `faster more than 250 times`!  
Thus, in presence of repeating queries, for quick handling it is better to preprocess our data first.

# Two laptop promotion check

Next, with promotion and give card with `precise sum for up to two laptops`, we need to check whether there are such two laptops (we can take both the same model).  
Adding method **check_promotion_dollars** to our Inventory class, where **dollars** is the exact sum of 1 or 2 laptops.

In [12]:
class Inventory():
    
    def __init__(self, csv_filename): # 'csv_filename.csv'
        with open(csv_filename) as open_f:
            file = list(csv.reader(open_f))
        self.header = file[0]
        self.rows = file[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
        self.id_to_row = {}
        self.prices = set()
        for row in self.rows:
            self.id_to_row[row[0]] = row
            self.prices.add(row[-1])
            
    def get_laptop_from_id(self, laptop_id): # 'laptop_id'
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id): # 'laptop_id'
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if dollars == row[-1]:
                return True
        for row1 in self.rows:
            for row2 in self.rows:
                if (row1[-1] + row2[-1]) == dollars:
                    return True
        return False

            
laptops = Inventory('laptops.csv')
print(f'It is possible to buy 1 or 2 laptops on 1000 $: {laptops.check_promotion_dollars(1000)}\nIt is possible to buy 1 or 2 laptops on 442 $: {laptops.check_promotion_dollars(442)}')

It is possible to buy 1 or 2 laptops on 1000 $: True
It is possible to buy 1 or 2 laptops on 442 $: False


# Optimization of two laptop promo check

To let our code run faster, let's implement **check_promotion_dollars_fast**, which will check the set instead of a list. The **self.prices** set was added to the init method.

In [13]:
class Inventory():
    
    def __init__(self, csv_filename): # 'csv_filename.csv'
        with open(csv_filename) as open_f:
            file = list(csv.reader(open_f))
        self.header = file[0]
        self.rows = file[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
        self.id_to_row = {}
        self.prices = set()
        for row in self.rows:
            self.id_to_row[row[0]] = row
            self.prices.add(row[-1])
            
    def get_laptop_from_id(self, laptop_id): # 'laptop_id'
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id): # 'laptop_id'
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if dollars == row[-1]:
                return True
        for row1 in self.rows:
            for row2 in self.rows:
                if (row1[-1] + row2[-1]) == dollars:
                    return True
        return False
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        for row in self.rows:
            if (dollars - row[-1]) in self.prices:
                return True
        return False
            
laptops = Inventory('laptops.csv')
print(f'It is possible to buy 1 or 2 laptops on 1000 $: {laptops.check_promotion_dollars_fast(1000)}\nIt is possible to buy 1 or 2 laptops on 442 $: {laptops.check_promotion_dollars_fast(442)}')

It is possible to buy 1 or 2 laptops on 1000 $: True
It is possible to buy 1 or 2 laptops on 442 $: False


# Comparing two methods

To measure the execution time of **check_promotion_dollars** and **check_promotion_dollars_fast** we again refer to time and random modules.

In [15]:
prices = [random.randint(100, 5000) for _ in range(100)]
laptops = Inventory('laptops.csv')

total_time_no_set = 0
for price in prices:
    start = time.time()
    laptops.check_promotion_dollars(price)
    end = time.time()
    total_time_no_set += end - start
    
total_time_set = 0
for price in prices:
    start = time.time()
    laptops.check_promotion_dollars_fast(price)
    end = time.time()
    total_time_set += end - start
    
print(f'Method without set, time: {total_time_no_set}\nMethod with set, time: {total_time_set}')

Method without set, time: 0.7590384483337402
Method with set, time: 0.0007834434509277344


Lookup in a set takes 0.001 second, while in a list - 1.5 second.  
Far more efficient in this case is to make a set from a list of lists - if lookup queries will be repeated.

# Binary search on laptops within a budget

Using binary search, we can find laptop that either be equal to our budget or first laptop over our budget.  
We can do this using sorted by price list - **self.rows_by_price** is added to the init method.  
Thus, we will find the index of the row with laptop info, which will be the first expensive one that we can afford. Or we'll get **-1** - if there are none of laptops in the budget.

In [10]:
class Inventory():
    
    def __init__(self, csv_filename): # 'csv_filename.csv'
        with open(csv_filename) as open_f:
            file = list(csv.reader(open_f))
        self.header = file[0]
        self.rows = file[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
        self.id_to_row = {}
        self.prices = set()
        for row in self.rows:
            self.id_to_row[row[0]] = row
            self.prices.add(row[-1])
        def row_price(row):
            return row[-1]
        self.rows_by_price = sorted(self.rows, key = row_price)
            
    def get_laptop_from_id(self, laptop_id): # 'laptop_id'
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id): # 'laptop_id'
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if dollars == row[-1]:
                return True
        for row1 in self.rows:
            for row2 in self.rows:
                if (row1[-1] + row2[-1]) == dollars:
                    return True
        return False
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        for row in self.rows:
            if (dollars - row[-1]) in self.prices:
                return True
        return False
    def find_first_laptop_more_expensive(self, price):
        start = 0
        end = len(self.rows_by_price) - 1
        if price < self.rows_by_price[-1][-1] and price >= self.rows_by_price[0][-1]:
            while start < end:
                middle = (start + end) // 2
                if self.rows_by_price[middle][-1] == price:
                    return middle
                elif self.rows_by_price[middle][-1] > price:
                    end = middle - 1
                else:
                    start = middle + 1
            return middle - 1
        elif price >= self.rows_by_price[-1][-1]:
            return len(self.rows_by_price) - 1
        else:
            return -1
        
laptops = Inventory('laptops.csv')
print(laptops.find_first_laptop_more_expensive(1000), laptops.find_first_laptop_more_expensive(10000))

682 1302


Now we know that `on 1 000 we can buy any laptop, starting from` the first (index 0) `till` 683 (index 682) `in our sorted data set`.  
And on 10 000 we can buy any of 1303 (last index 1302) models.

# Findings

Lookup in a set takes 0.001 second, while in a list - 1.5 second.  
Far more efficient in this case is to make a set from a list of lists - if lookup queries will be repeated.